In [87]:
import numpy as np
from scipy.stats import ortho_group

# Input vector

In [88]:
y = np.array([1, 2, 3, 4, 5])

# Initialization

In [89]:
N = 3          #   No. of basis functions
K = len(y)     #   Dimensions of the input vector
gamma = 6
tol = 1e-5
active_set = set()
features = np.zeros(N)
signs = np.zeros(N)
A = ortho_group.rvs(dim=K)    # Matrix containing orthogonal bases
A = A[:,:N]
part_diff = np.zeros(N)

In [97]:
print('Basis vectors are: \n' + str(A))

Basis vectors are: 
[[ 0.6273561   0.51992779  0.07634933]
 [ 0.36607971 -0.46775075 -0.20412154]
 [-0.35940133  0.59977998 -0.60359217]
 [ 0.57128414  0.0177932  -0.47048882]
 [ 0.12990412  0.38837059  0.60565827]]


# Feature sign search algorithm

In [90]:
while True:
    for i in range(N):
        for j in range(K):
            part_diff[i] += -2*(y[j] - A[j, :].dot(features))*A[j, i]
    max_val = np.amax(part_diff)
    max_ind = np.argmax(np.abs(part_diff))
    if(max_val > gamma):
        signs[max_ind] = -1
        active_set = active_set | {max_ind}
    if(max_val < -gamma):
        signs[max_ind] = 1
        active_set = active_set | {max_ind}

    while True:
        A_hat = A[:, list(active_set)]
        features_hat = features[list(active_set)]
        signs_hat = signs[list(active_set)]
        features_hat_new = np.linalg.inv(A_hat.T @ A_hat) @ ((A_hat.T @ y) - (gamma/2)*signs_hat)
        features_intermediate = np.zeros((len(active_set), 20))
        features_intermediate[:,0] = features_hat
        features_min = features_hat_new
        obj_min = ((y - A_hat @ features_min).T @ (y - A_hat @ features_min)) + gamma*(signs_hat.T @ features_min)
        for i in range(1, 20):
            features_intermediate[:,i] = features_hat + (features_hat_new - features_hat)*i/20
            if(np.any(features_intermediate[:,i]*features_intermediate[:,i-1] < 0)):
                obj_intermediate = ((y - A_hat @ features_intermediate[:,i]).T @ (y - A_hat @ features_intermediate[:,i])) + gamma*(signs_hat.T @ features_intermediate[:,i])
                if(obj_intermediate < obj_min):
                    features_min = features_intermediate[:,i]
                    obj_min = obj_intermediate

        features_hat = features_min
        features[list(active_set)] = features_hat
        for i in range(len(features_hat)):
            if(np.abs(features_hat[i]) < tol):
                active_set.remove(list(active_set)[i])
        signs = np.sign(features)

        flag_1 = 0
        for i in range(N):
            if(np.abs(features[i]) > tol):
                p_d = 0
                for j in range(K):
                    p_d += -2*(y[j] - A[j, :].dot(features))*A[j, i]
                if(abs(p_d + gamma*signs[i]) > tol):
                    flag_1 = 1
                    break
        if(flag_1 == 0):
            break
    
    flag_2 = 0
    for i in range(N):
        if(np.abs(features[i]) < tol):
            p_d = 0
            for j in range(K):
                p_d += -2*(y[j] - A[j, :].dot(features))*A[j, i]
            if(abs(p_d) > gamma):
                flag_2 = 1
                break
    if(flag_2 == 0):
        break

In [94]:
print('The active set consists of: '+ str(active_set))

The active set consists of: {0, 1}


In [102]:
print('Coefficients (or features): '+ str(features))

Coefficients (or features): [0.21596869 0.39679194 0.        ]
